In [24]:
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator
from qiskit import transpile
from qiskit.visualization import plot_histogram
from qiskit_aer import AerSimulator
import numpy as np
import matplotlib.pyplot as plt

In [1]:
def oracle(n):
    qc = QuantumCircuit(n)
    qc.cz(0, 1)
    return qc
n = 2
qc = QuantumCircuit(n, n)
qc.h(range(n))
iterations = int(np.pi / 4 * np.sqrt(2**n))
for _ in range(iterations):
    qc.append(oracle(n), range(n))
    qc.h(range(n))
    qc.x(range(n))
    qc.h(n-1)
    qc.cx(0, 1)
    qc.h(n-1)
    qc.x(range(n))
    qc.h(range(n))

qc.measure(range(n), range(n))
simulator = AerSimulator()
transpiled_qc = transpile(qc, simulator)
job = simulator.run(transpiled_qc, shots=1024)
result = job.result()
counts = result.get_counts()
print("Grover's Algorithm Results:")
print(counts)

NameError: name 'QuantumCircuit' is not defined